## Import Libraries

In [ ]:
# Import necessary libraries for data handling, optimization, and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import pickle
from itertools import permutations
from gurobipy import *
from gurobipy import GRB

In [ ]:
# Set your working directory
my_folder = "C:/Users/hyunwoolee/OneDrive - Virginia Tech/Hyunwoo Research"

## General Functions

In [ ]:
# Converts a set of selected lakes into selfish and global utility values.
# - 'selected_lakes' is a list of lake indices selected by the players (counties).
# - Returns: 
#     1. A dictionary of selfish utilities per county.
#     2. A scalar value representing the global utility (social welfare).

def from_x_to_obj(selected_lakes):
    
    # fix x-variables based on selected_lakes
    x = {i: 1 if i in selected_lakes else 0 for i in I}
    y = {arc: min(1,x[arc[0]]+x[arc[1]]) for arc in arcs}
    
    # Compute selfish objectives for each county
    selfish_obj = {county: sum(n[arc] * y[arc] *t [arc] for arc in arcs if arc[1] in I_c[county]) for county in counties}

    # Compute global objective 
    global_obj = sum(n[arc]*y[arc]*t[arc] for arc in arcs)   

    return selfish_obj, global_obj

In [ ]:
def printSolution(model, x, selected_lakes):
    
    # Check if the model found a feasible solution
    if model.SolCount > 0:
        print('\nObjective value: %g' % model.ObjVal)

        # Append selected lakes (x[i] ≈ 1) to the result list
        for i in I:
            if x[i].X > 0.1:
                # print('Lake %s: %g' % (i, x[i].X))
                selected_lakes.append(i)
    else:
        pass  # No feasible solution found


## Load Minnesota Dataset

In [ ]:
# ======================= Minnesota Dataset ======================= #

dataset = 'Minnesota'
user_Threads = 16

# Load data
df_edge = pd.read_csv(my_folder + "/BZR_EBMC/EBMC_minnesota/edgelist1.csv")    
budget_count = pd.read_excel(my_folder + "/BZR_EBMC/EBMC_minnesota/Number of inspection stations.xlsx", header=1)    

# --- Preprocessing for edge data ---
counties = np.unique(np.concatenate((df_edge['county_name.origin'].unique(), df_edge['county_name.destination'].unique()), axis=0))
county_names = counties
lakes = np.unique(np.concatenate((df_edge['dow_origin'].unique(), df_edge['dow_destination'].unique()), axis=0))

lake_county = {}
for lake in lakes:
    if len(df_edge[df_edge['dow_origin'] == lake]) > 0:
        lake_county[lake] = df_edge[df_edge['dow_origin'] == lake]['county_name.origin'].iloc[0]
    else:
        lake_county[lake] = df_edge[df_edge['dow_destination'] == lake]['county_name.destination'].iloc[0]

county_lakes = {county: [] for county in county_names}
for lake in lakes:
    origin_county = df_edge[df_edge['dow_origin'] == lake]['county_name.origin']
    if not origin_county.empty:
        county_lakes[origin_county.iloc[0]].append(lake)
    else:
        dest_county = df_edge[df_edge['dow_destination'] == lake]['county_name.destination'].iloc[0]
        county_lakes[dest_county].append(lake)

# --- Preprocessing for budget data ---
budget_count = budget_count.drop(columns='Unnamed: 0')
budget_count['Number of inspection stations'] = round(budget_count['Number of inspection stations'], 0)
budget_count['County'] = budget_count['County'].str.lower()

county_budget = {}
for county in county_names:
    num = int(budget_count[budget_count['County'] == county]['Number of inspection stations'].values[0])
    county_budget[county] = num

# --- Infested lakes ---
infested_lakes = df_edge[
    (df_edge['zm2019.origin'] == 1) |
    (df_edge['ss2019.origin'] == 1) |
    (df_edge['ew2019.origin'] == 1) |
    (df_edge['sf2019.origin'] == 1)
]['dow_origin'].unique()

num_infested_lakes = {county: 0 for county in counties}
for lake in infested_lakes:
    if lake in lake_county:
        num_infested_lakes[lake_county[lake]] += 1

# --- Weight calculation ---
w = {i: 0 for i in lakes}
for _, row in df_edge.iterrows():
    if row['bij'] != 0:
        w[row['dow_origin']] += row['bij'] * row['weight']
        w[row['dow_destination']] += row['bij'] * row['weight']

# --- Parameter settings for models ---
I = lakes
I_c = {c: [i for i in I if lake_county[i] == c] for c in counties}
I_c_complement = {c: [i for i in I if i not in I_c[c]] for c in counties}

n = {(row['dow_origin'], row['dow_destination']): row['weight'] for _, row in df_edge.iterrows()}
t = {(row['dow_origin'], row['dow_destination']): row['bij'] for _, row in df_edge.iterrows()}
arcs = list(n.keys())

arcs_c = {c: [a for a in arcs if a[0] in I_c[c] and a[1] in I_c[c]] for c in counties}
arcs_plus_c = {c: [a for a in arcs if a[1] in I_c[c] and a[0] in I_c_complement[c]] for c in counties}
arcs_minus_c = {c: [a for a in arcs if a[0] in I_c[c] and a[1] in I_c_complement[c]] for c in counties}


## Non-game strategy profile

In [ ]:
# === Non-game (NG) Strategy Profile Construction ===
# In this setting, each county individually maximizes its own utility assuming other counties take no action.
# That is, a county solves its best-response problem while all others remain inactive.
# This gives us a baseline utility level for each county independent of game-theoretic interactions.

def selfish_non(county):
    """
    Solves a single-county optimization problem under the Non-game (NG) setting.

    In this model, only the given county is allowed to make decisions while all other counties are restricted
    from selecting any lakes. This simulates a scenario where the player acts unilaterally.

    Args:
        county (str): The county (player) under consideration.

    Returns:
        list: A list of selected lakes for the county in the non-game setting.
    """

    # Initialize a Gurobi model
    model = Model("Selfish_NonGame")
    model.Params.LogToConsole = 0
    model.Params.MIPGap = 1e-6
    model.setParam('TimeLimit', 60)
    model.Params.Threads = user_Threads

    # Binary decision variables: x[i] = 1 if lake i is selected
    x = model.addVars(I_c[county] + I_c_complement[county], vtype=GRB.BINARY, name="x")

    # Arc variables: u[arc] = 1 if arc is activated (both endpoints selected)
    relevant_arcs = [arc for arc in arcs if arc[1] in I_c[county]]
    u = model.addVars(relevant_arcs, vtype=GRB.BINARY, name="u")

    # Budget constraint for the given county
    model.addConstr(quicksum(x[i] for i in I_c[county]) <= county_budget[county])

    # Arc linkage constraints: ensure arc activation is consistent with selected lakes
    model.addConstrs((u[arc] <= x[arc[0]] + x[arc[1]] for arc in relevant_arcs), name="ArcLink")

    # Fix decisions for all non-county lakes to 0 (other counties do nothing)
    model.addConstrs((x[i] == 0 for i in I_c_complement[county]), name="OtherPlayersFixed")

    # Objective: maximize utility derived from activated arcs
    model.setObjective(quicksum(t[arc] * n[arc] * u[arc] for arc in relevant_arcs), GRB.MAXIMIZE)

    # Solve the model
    model.optimize()
    
    # Extract selected lakes for this county
    selected_lakes = [i for i in I_c[county] if x[i].X > 0.1]

    return selected_lakes


# === Run NG Model for All Counties ===
print("="*35 + "  NG_model  " + "="*35)

# Run non-game optimization individually for each county
selected_lakes = {}
for county in counties:
    selected_lakes[county] = selfish_non(county)

# Flatten all county-level solutions into a unified strategy profile
NG_selected_lakes = [lake for key, val in selected_lakes.items() for lake in val]


## Different Strategy Profiles

In [ ]:
# === County-Level Strategy Profile Comparison ===
# In this section, we compare utility outcomes across multiple strategic solutions for the Minnesota EBMC instance.
#
# The following strategy profiles are analyzed:
#   - First PNE identified by the BZR algorithm        → `first_PNE`
#   - Best PNE (highest global utility) from BZR       → `best_PNE`
#   - Optimal Social Welfare solution from OSW model   → `OSW`
#   - Non-game strategy profile (each county acts selfishly, assuming others do nothing) → `NG`
#
# Note:
# - `first_PNE`, `best_PNE`, and `OSW` are imported from prior computational experiments (see `BZR_EBMC.ipynb`).
# - `NG` is computed directly within this file using the `selfish_non()` function (see above).

# --- Strategy Profiles ---

# First Pure Nash Equilibrium identified
first_PNE = [1003300, 1006200, 1008900, 1009300, 1013600, 1014200, 1014700, 1015700, 1015900, 1016100, 1020400, 1020900, 
             2000800, 2001300, 2001600, 2008900, 2009100, 2013000, 3001000, 3013600, 3019500, 3024100, 3028600, 3028700, 
             3035900, 3038100, 3038300, 3038700, 3047500, 3050000, 3057600, 3058200, 3058800, 4001100, 4003000, 4003500, 
             4004900, 4007900, 4013000, 4013500, 4015900, 5001300, 6000100, 6000200, 6013800, 7004300, 7005300, 7007700, 
             7009600, 7009800, 8002600, 8004500, 9003800, 9004100, 9006000, 10000900, 10005900, 10008900, 10009300, 10009400, 
             10016100, 11004300, 11005900, 11006200, 11014200, 11014300, 11014700, 11016700, 11017100, 11017400, 11020000, 11020100,
             11020300, 11025000, 11030400, 11030500, 11030700, 11030800, 11041100, 11041300, 11041500, 11050400, 12004500, 13001200, 
             13003100, 13003300, 13004100, 13006900, 13007300, 14000400, 15001600, 15005700, 15006800, 15014900, 16000100, 16002700, 
             16002900, 16004100, 16007700, 16014300, 16034800, 16035600, 16063300, 17000700, 17006000, 18002000, 18003400, 18003800, 
             18004100, 18008800, 18009000, 18009600, 18013600, 18014500, 18020300, 18021200, 18024200, 18024300, 18026100, 18030500, 
             18030800, 18031000, 18031100, 18031200, 18031500, 18032000, 18035200, 18036100, 18037200, 18037300, 18037400, 18037500, 
             18037800, 18037900, 18040300, 18041200, 19000600, 19002000, 19002700, 19005200, 19008300, 21005200, 21005600, 21005700, 
             21007900, 21008300, 21009200, 21010600, 21012300, 21014400, 21014500, 21018000, 22007400, 24001800, 24006700, 25000100,
             25001800, 25001900, 26000200, 26009700, 27001600, 27003100, 27006700, 27010400, 27012500, 27012700, 27012800, 27013200, 
             27013300, 27013700, 27016500, 27017600, 27018800, 29006100, 29007100, 29007700, 29014600, 29015100, 29015600, 29016100, 
             29018500, 29021600, 29024300, 29025000, 30008000, 30010700, 30013800, 31006700, 31019000, 31021600, 31033400, 31035300, 
             31038400, 31039200, 31041000, 31053200, 31053300, 31053800, 31055400, 31057600, 31065300, 31071700, 31071900, 31072200, 
             31072500, 31078600, 31081200, 31081300, 31082600, 31085000, 31085300, 31085700, 31088200, 31089600, 31091300, 32001800, 
             32002200, 32006900, 33003600, 33004000, 34004400, 34006600, 34007900, 34008600, 34014200, 34015400, 34015800, 34016900, 
             34017100, 34018000, 34019200, 35000300, 36000100, 36000900, 36001100, 36001800, 36002400, 37004600, 37010700, 38002400, 
             38052900, 38064500, 38077900, 38078200, 38078400, 38081100, 38081700, 39000200, 39000500, 40000200, 40001000, 40001400, 
             40002800, 40010900, 40011700, 41004300, 41008900, 41011000, 42004700, 42005200, 43008400, 44001400, 46001000, 46005100, 
             46010900, 47000200, 47001500, 47003200, 47003800, 47004600, 47006200, 47010200, 47011900, 47015300, 47017700, 48000200, 
             48000900, 48001200, 48003600, 49001600, 49007900, 49008000, 49008100, 49013700, 50000400, 51000600, 51002400, 51004000,
             51004600, 52001500, 53002100, 53002800, 54000500, 56013000, 56013800, 56014100, 56021400, 56023900, 56024200, 56031000, 
             56032800, 56036000, 56038300, 56038500, 56038600, 56052300, 56061300, 56074700, 56074900, 56076000, 56078100, 56078600, 
             56094500, 58004800, 58006200, 58006700, 58012300, 58013800, 60006900, 60021700, 60030500, 61006400, 61007800, 61013000, 
             61018000, 62000200, 62000400, 62007000, 62007800, 64005800, 65000600, 66000800, 66001000, 66001800, 66003200, 68000400, 
             69000300, 69000400, 69006900, 69011500, 69011800, 69012100, 69022400, 69028500, 69037200, 69037500, 69037800, 69047000, 
             69048900, 69049000, 69049100, 69049800, 69051100, 69060800, 69061600, 69061700, 69066000, 69069300, 69069400, 69074400, 
             69084100, 69084200, 69084500, 69084800, 69086400, 69093900, 70005400, 70008700, 70009100, 71001600, 71014100, 71014600, 
             72001300, 72005000, 72008900, 73005500, 73010600, 73011700, 73013300, 73013900, 73014700, 73015700, 73015900, 73019600, 
             73020000, 74002300, 75007500, 76007200, 77000700, 77002300, 77008400, 77015400, 77016400, 77018100, 77021500, 78002500, 
             79000500, 79000600, 79001400, 80003000, 80003400, 81006700, 81009500, 82004900, 82005200, 82005400, 82014000, 82015900, 
             82016700, 83004000, 85000300, 85001100, 85001300, 85003300, 86009000, 86013400, 86022700, 86023300, 86025100, 86025200, 
             86027900, 86028800, 86028900, 86029300, 87001600, 87003000]

# Best Pure Nash Equilibrium (with highest global utility)
best_PNE = [1003300, 1006200, 1008900, 1009300, 1013600, 1014200, 1014700, 1015700, 1015900, 1016100, 1020400, 1020900, 2000800, 
            2001300, 2001600, 2008900, 2009100, 2013000, 3001000, 3013600, 3019500, 3024100, 3028600, 3028700, 3035900, 3038100, 
            3038300, 3038700, 3047500, 3050000, 3057600, 3058200, 3058800, 4001100, 4003000, 4003500, 4004900, 4007900, 4013000, 
            4013500, 4015900, 5001300, 6000100, 6000200, 6013800, 7004300, 7005300, 7007700, 7009600, 7009800, 8002600, 8004500, 
            9003800, 9004100, 9006000, 10000900, 10005900, 10008900, 10009300, 10009400, 10016100, 11004300, 11005900, 11006200, 
            11014200, 11014300, 11014700, 11016700, 11017100, 11017400, 11020000, 11020100, 11020300, 11025000, 11030400, 11030500, 
            11030700, 11041100, 11041200, 11041300, 11041500, 11050400, 12004500, 13001200, 13003100, 13003300, 13004100, 13006900, 
            13007300, 14000400, 15001600, 15005700, 15006800, 15014900, 16000100, 16002700, 16002900, 16004100, 16007700, 16014300, 
            16034800, 16035600, 16063300, 17000700, 17006000, 18002000, 18003400, 18003800, 18004100, 18008800, 18009000, 18009600,
            18013600, 18014500, 18020300, 18021200, 18024200, 18024300, 18026100, 18030500, 18030800, 18031000, 18031100, 18031200, 
            18031500, 18032000, 18035200, 18036100, 18037200, 18037300, 18037400, 18037500, 18037800, 18037900, 18040300, 18041200,
            19000600, 19002000, 19002700, 19005200, 19008300, 21005200, 21005600, 21005700, 21007900, 21008300, 21009200, 21010600,
            21012300, 21014400, 21014500, 21018000, 22007400, 24001800, 24006700, 25000100, 25001800, 25001900, 26000200, 26009700, 
            27001600, 27003100, 27006700, 27010400, 27011800, 27012500, 27012700, 27012800, 27013200, 27013300, 27013700, 27017600, 
            27018800, 29006100, 29007100, 29007700, 29014600, 29015100, 29015600, 29016100, 29018500, 29021600, 29024300, 29025000, 
            30008000, 30010700, 30013800, 31006700, 31019000, 31021600, 31033400, 31035300, 31038400, 31039200, 31041000, 31053200, 
            31053300, 31053800, 31055400, 31057600, 31065300, 31071700, 31071900, 31072200, 31072500, 31078600, 31081200, 31081300, 
            31082600, 31085000, 31085300, 31085700, 31088200, 31089600, 31091300, 32001800, 32002200, 32006900, 33003600, 33004000, 
            34004400, 34006600, 34007900, 34008600, 34014200, 34015400, 34015800, 34016900, 34017100, 34018000, 34019200, 35000300, 
            36000100, 36000900, 36001100, 36001800, 36002400, 37004600, 37010700, 38002400, 38052900, 38064500, 38077900, 38078200, 
            38078400, 38081100, 38081700, 39000200, 39000500, 40000200, 40001000, 40001400, 40002800, 40010900, 40011700, 41004300, 
            41008900, 41011000, 42004700, 42005200, 43008400, 44001400, 45011900, 46001000, 46005100, 46010900, 47000200, 47001500, 
            47003200, 47003800, 47004600, 47006200, 47010200, 47011900, 47015300, 47017700, 48000200, 48000900, 48001200, 48003600, 
            49001600, 49007900, 49008000, 49008100, 49013700, 50000400, 51000600, 51002400, 51004000, 51004600, 52001500, 53002100, 
            53002800, 54000500, 56013000, 56013800, 56014100, 56021400, 56023900, 56024200, 56031000, 56032800, 56036000, 56038300, 
            56038500, 56038600, 56052300, 56061300, 56074700, 56074900, 56076000, 56078100, 56078600, 56094500, 57001300, 58004800, 
            58006200, 58006700, 58012300, 58013800, 60006900, 60021700, 60030500, 61006400, 61007800, 61013000, 61018000, 62000200, 
            62000400, 62007000, 62007800, 64005800, 65000200, 66000800, 66001000, 66001800, 66003200, 68000400, 69000300, 69000400, 
            69006900, 69011500, 69011800, 69012100, 69022400, 69028500, 69037200, 69037500, 69037800, 69047000, 69048900, 69049000, 
            69049100, 69049800, 69051100, 69060800, 69061600, 69061700, 69066000, 69069300, 69069400, 69074400, 69084100, 69084200,
            69084500, 69084800, 69086400, 69093900, 70005400, 70008700, 70009100, 71001600, 71014100, 71014600, 72001300, 72005000, 
            72008900, 73005500, 73010600, 73011700, 73013300, 73013900, 73014700, 73015700, 73015900, 73019600, 73020000, 74002300, 
            75007500, 76007200, 77000700, 77002300, 77008400, 77015400, 77016400, 77018100, 77021500, 78002500, 79000500, 79000600, 
            79001400, 79002100, 80003000, 80003400, 81006700, 81009500, 82004900, 82005200, 82005400, 82014000, 82015900, 82016700, 
            83004000, 85000300, 85001100, 85001300, 85003300, 86009000, 86013400, 86022700, 86023300, 86025100, 86025200, 86027900, 
            86028800, 86028900, 86029300, 87001600, 87003000]

# Optimal Social Welfare strategy
OSW = [1003300, 1006200, 1008900, 1009300, 1010400, 1013600, 1014200, 1014700, 1015700, 1015900, 1020400, 1020900, 2000400, 2000600, 
       2002600, 2004200, 2008400, 2009100, 3001000, 3019500, 3024100, 3028600, 3028700, 3035900, 3038100, 3038700, 3047500, 3050000, 
       3050300, 3050600, 3057600, 3058200, 3058800, 4001100, 4003000, 4003500, 4003800, 4007900, 4013000, 4013500, 4015900, 5001300, 
       6000100, 6000200, 6013800, 7004400, 7005400, 7006000, 7009600, 7009800, 8002600, 8004500, 9000800, 9003400, 9003500, 10000200, 
       10000600, 10000900, 10001500, 10001900, 10005900, 11004300, 11005900, 11006200, 11014300, 11014700, 11016700, 11017400, 11019000, 
       11020100, 11020300, 11024200, 11025000, 11030400, 11030500, 11030700, 11038300, 11041100, 11041200, 11041300, 11041500, 11050400, 
       12004500, 13001200, 13002700, 13004100, 13005300, 13006800, 13006900, 14010000, 15001600, 15005700, 15008100, 15014900, 16000100, 
       16002700, 16002900, 16004100, 16004900, 16007700, 16014300, 16014700, 16063300, 17000700, 17006000, 18003400, 18003800, 18004100, 
       18009000, 18009600, 18014500, 18018500, 18020300, 18021200, 18024200, 18024300, 18026100, 18026600, 18030500, 18030800, 18031000, 
       18031100, 18031200, 18031500, 18032000, 18035200, 18035400, 18036100, 18037200, 18037300, 18037400, 18037500, 18037800, 18037900, 
       18040300, 18041200, 19000500, 19000600, 19002600, 19002700, 19003100, 21005200, 21005400, 21005600, 21005700, 21008300, 21009000, 
       21009200, 21010600, 21012300, 21014400, 21025700, 22007400, 24001800, 24006700, 25000100, 25001700, 25001900, 26009500, 26009700, 
       27000300, 27001600, 27001900, 27003100, 27006700, 27010400, 27011700, 27011800, 27013300, 27013700, 27016000, 27017600, 27018400, 
       29003600, 29006100, 29007500, 29014600, 29015100, 29015600, 29016100, 29018500, 29020800, 29021600, 29024300, 30004300, 30007200, 
       30013500, 31006700, 31019000, 31019100, 31021600, 31035300, 31037000, 31037200, 31037300, 31038400, 31039200, 31053200, 31053300, 
       31055400, 31057600, 31065300, 31071900, 31072200, 31072500, 31078600, 31081200, 31081300, 31082600, 31085000, 31085300, 31085700, 
       31088200, 31089600, 31091300, 32001800, 32002200, 32006900, 33002800, 33004000, 34004400, 34006200, 34007900, 34011900, 34014200, 
       34015400, 34017100, 34020600, 34021700, 34022400, 34025100, 35000300, 36000100, 36000900, 36001100, 36001800, 36002400, 37004600, 
       37010700, 38002400, 38052900, 38064500, 38077900, 38078200, 38078400, 38081100, 38081700, 39000200, 39000500, 40003100, 40005600, 
       40005700, 40006300, 40009200, 40011700, 41004300, 41008900, 41011000, 42004700, 42005200, 43011500, 44001400, 45011900, 46004900, 
       46005100, 46010900, 47001600, 47004600, 47004900, 47005000, 47006400, 47006800, 47009500, 47009600, 47011900, 47013400, 48000200, 
       48000900, 48001200, 48003600, 49001600, 49007900, 49012700, 49013300, 49013700, 50000400, 51002400, 51002700, 51004600, 51006800, 
       52001500, 53002100, 53002800, 54000500, 56003100, 56013800, 56014100, 56021400, 56023900, 56024200, 56025300, 56032800, 56033500, 
       56036000, 56038600, 56038700, 56052300, 56061300, 56074700, 56076000, 56078100, 56078600, 56082400, 56094500, 57001300, 58006700, 
       58008100, 58011900, 58013800, 58014200, 60003200, 60021700, 60030500, 61006400, 61007800, 61013000, 61018000, 62000200, 62006100, 
       62006700, 62007800, 64005800, 65000600, 66003800, 66003900, 66005200, 66005500, 68000400, 69000300, 69000400, 69006900, 69011500, 
       69011800, 69012100, 69016700, 69017400, 69022400, 69028500, 69037200, 69037500, 69037800, 69047000, 69048900, 69049000, 69049100, 
       69049800, 69060800, 69061600, 69061700, 69066000, 69069300, 69069400, 69084100, 69084200, 69084500, 69086400, 69091400, 69093900, 
       70002600, 70007200, 70009500, 71005500, 71006700, 71008200, 72001300, 72005000, 72008900, 73001400, 73005500, 73007000, 73010600, 
       73012300, 73012800, 73013300, 73019600, 73020000, 73024900, 74002300, 75007500, 76007200, 77000700, 77001900, 77008400, 77008900, 
       77015000, 77016400, 77021500, 78002500, 79000500, 79000600, 79001400, 79002100, 80003000, 80003400, 81001400, 81005500, 82005200,
       82005400, 82010600, 82015900, 82016300, 82016700, 83004000, 85000100, 85000200, 85001100, 85001300, 86002300, 86005300, 86009000, 
       86013400, 86022700, 86023300, 86025100, 86025200, 86027900, 86028800, 87001500, 87003000]

# Non-game solution (computed locally in this notebook)
NG = NG_selected_lakes

# --- Compute Utility Scores ---

# The function `from_x_to_obj()` computes:
#   - selfish_obj: a dictionary of utilities for each county
#   - global_obj: system-wide utility (i.e., total social welfare)

selfish_obj_first_PNE, global_obj_first_PNE = from_x_to_obj(first_PNE)
selfish_obj_best_PNE, global_obj_best_PNE = from_x_to_obj(best_PNE)
selfish_obj_OSW, global_obj_OSW = from_x_to_obj(OSW)
selfish_obj_NG, global_obj_NG = from_x_to_obj(NG)

## Generate County-level Output

In [ ]:
# === Generate County-Level Output Table ===
# This section aggregates and compares utility values across strategy profiles 
# (NG, first_PNE, best_PNE, OSW) for each county, as well as totals at the bottom row.

# Construct the DataFrame with utility values per county
# - Each row corresponds to a county
# - Columns include number of infested lakes, budget, and utility values under each strategy
data = {
    'infested_lakes': num_infested_lakes,
    'budget': county_budget,
    'NG': selfish_obj_NG,
    'first_PNE': selfish_obj_first_PNE,
    'best_PNE': selfish_obj_best_PNE,
    'OSW': selfish_obj_OSW
}
df = pd.DataFrame(data)

# Append a "total" row summarizing overall statistics:
# - Total number of infested lakes
# - Total budget across counties
# - Global utility values under each strategy
new_row = {
    'infested_lakes': len(infested_lakes),
    'budget': sum(county_budget[county] for county in counties),
    'NG': global_obj_NG,
    'first_PNE': global_obj_first_PNE,
    'best_PNE': global_obj_best_PNE,
    'OSW': global_obj_OSW
}
df = pd.concat([df, pd.DataFrame([new_row], index=['total'])], ignore_index=False)

In [ ]:
# Display the resulting DataFrame
df

In [ ]:
# Export to CSV for reporting
df.to_csv(my_folder + "/BZR_EBMC/EBMC_results/minnesota/county-level.csv")